In [66]:
import pandas as pd
import numpy
import json

In [67]:

def vek(vstup):
    try:
        return int(abs(float(vstup)))/1000 if abs(float(vstup)) > 1000 else int(abs(float(vstup)))
    except ValueError:
        return numpy.nan
    

konvertovace = {
    'age' : vek
}

personalne = pd.read_csv("data/personal_train.csv", na_values='?', index_col=0, converters=konvertovace, )
personalne = personalne.drop(columns='date_of_birth')

Nacitane personalne data, s opravenym vekom, odstranenym datumom narodenia (nie je potrebny)

In [68]:
bulo = lambda vstup: numpy.nan if pd.isna(vstup) or len(vstup) is 0 else str(vstup).lower().split('.')[0][0] == 't'
classa = lambda vstup: vstup.lower().split('.|')[0]


def js(vstup):
    try:
        return str(vstup).replace("'", '"')
    except json.JSONDecodeError:
        pass
konvertovace = {
    "FTI measured": bulo,
    "tumor": bulo,
    "lithium": bulo,
    "thyroid surgery": bulo,
    "I131 treatment": bulo,
    "goitre": bulo,
    "hypopituitary": bulo,
    "query on thyroxine": bulo,
    "T3 measured": bulo,
    "TBG measured": bulo,
    "pregnant": bulo,
    "class": classa,
    "medical_info": js
}
other = pd.read_csv("data/other_train.csv", na_values=['?', '??', ' ?'], index_col=0, true_values=['t'],
                    false_values=['f'],
                    converters=konvertovace)
other = other.drop(columns=['TBG'])


Nacitane ostatne udaje s opravenymi boolovskymi hodnotami a vyhodenym prazdnym stlpcom TBG

In [69]:
def rozober_json(vstup):
    try:
        return pd.Series(json.loads(vstup))
    except json.JSONDecodeError:
        return pd.Series(json.loads('{}'))
other = other.join(other['medical_info'].apply(rozober_json))
other = other.drop(columns='medical_info')

Rozoberanie jsonu v medical_info stlpci na jednotlive stlpce

In [70]:
for stlpec in ['TSH measured', 'query hyperthyroid', 'TT4 measured', 'on antithyroid medication', 'sick', 'T4U measured', 'psych', 'query hypothyroid']:
    other[stlpec] = other[stlpec].apply(bulo)

other = other.replace('?', numpy.nan)

Oprava pridanych boolovskych stlpcov a nahradenie ? za NaN

In [71]:
vsetko = other.merge(personalne, on=('name', 'address'))
vsetko = vsetko.replace(to_replace=[None], value=numpy.nan)

Spojenie suborov a nahradenie None za NaN

In [72]:
def agreguj(vstup):
    vstup = vstup.reset_index()
    for hodnota in vstup.values:
         if not pd.isna(hodnota[1]):
             return hodnota[1]
    return numpy.nan
        
vsetko = vsetko.groupby(['name', 'address']).aggregate(agreguj).reset_index()

Spojenie rovnakych riadkov

In [73]:
numericke = vsetko.select_dtypes(include=numpy.number).columns
mediany = {}
for stlpec in numericke:
    mediany[stlpec] = vsetko[stlpec].median()
vsetko = vsetko.replace(numpy.nan, value=mediany)

In [74]:
vsetko['TSH measured'].unique()

In [75]:
vsetko.FTI = pd.to_numeric(vsetko.FTI)

In [76]:
vsetko.goitre.unique()

array([False,  True])